In [1]:
import os
os.getcwd()

'/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/notebooks'

In [5]:
import sys
sys.path.append('../')
sys.path.append('..')

In [4]:
sys.path

['/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/venv/lib/python3.10/site-packages',
 '../']

In [8]:
data = pd.read_csv('../pytorch_models/Data/daum/daum_movie.csv')

In [9]:
data

,Unnamed: 0,nickName,movieId,rating,timestamp,userId,review
0,0,매력적그녀,121721,8,1.702394e+09,NaN,이런 영화 안좋아하는데 어쩔수 없이 봤다가 솔직히 기분은 진짜 별로였음 근데 또 영...
1,1,매력적그녀,1660,10,1.701700e+09,NaN,첨엔 내가 주인공이고 이렇게 술술 풀리는 행복한 인생 부럽다 이러다가 갈수록 숨막히...
2,2,매력적그녀,164920,10,1.692027e+09,NaN,처음엔 넷플에.떴길래 별 생각없이 보기 시작했는데 너무 잔잔해서 뭐지 하다가 이상하...
3,3,매력적그녀,130710,1,1.684073e+09,NaN,이거 왜 영화로 만든건가요?
4,4,매력적그녀,1953,6,1.677165e+09,NaN,당시엔 평이 엄청 낮고 여주 혹평이던 영화도 시간이 지나면 이렇게 평점이 좋아지는구...
...,...,...,...,...,...,...,...
582312,582312,아림,96030,10,1.614238e+09,NaN,👍👍👍👍👍👍
582313,582313,아림,134684,9,1.593245e+09,NaN,사람들마다 취향이 다르니 재미없으신 분들도 계시겠지만 부산행+엑시트 느낌이라 재밌었...
582314,582314,버들치,48253,10,1.482238e+09,NaN,가슴이 아프다..
582315,582315,이뿐수기,45801,9,1.516678e+09,NaN,추자현하면 이 영화가 생각남


In [10]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import time
import math
import os
import sys

# data = pd.read_csv('pytorch_models/Data/daum/daum_movie.csv')
data.columns = ['unnamed', 'uid', 'iid', 'r', 'ts', 'nan', 'review']
data = data.drop(columns=['unnamed', 'ts', 'nan', 'review'])

In [11]:
data

,uid,iid,r
0,매력적그녀,121721,8
1,매력적그녀,1660,10
2,매력적그녀,164920,10
3,매력적그녀,130710,1
4,매력적그녀,1953,6
...,...,...,...
582312,아림,96030,10
582313,아림,134684,9
582314,버들치,48253,10
582315,이뿐수기,45801,9


In [13]:
data['uid'].nunique()

35813

In [14]:
# %% rating이 5개 미만인 사용자 제거
gb_inum = data[['uid', 'iid']].groupby(['uid']).count()
over_20_idxs = gb_inum.loc[gb_inum.iid > 5].index.values
data = data.loc[data.uid.isin(over_20_idxs)].reset_index(drop=True)

# %% uid, iid encoding
user_encoder = {id: idx for idx, id in enumerate(data['uid'].unique())}
user_decoder = {idx: id for id, idx in user_encoder.items()}

item_encoder = {id: idx for idx, id in enumerate(data['iid'].unique())}
item_decoder = {idx: id for id, idx in item_encoder.items()}

data['uid'] = data['uid'].apply(lambda x: user_encoder[x])
data['iid'] = data['iid'].apply(lambda x: item_encoder[x])

In [18]:
len(user_encoder)

15038

In [17]:
max(user_encoder.values())

15037

In [21]:
max(item_encoder.values())

29712

In [22]:
len (item_encoder)

29713

In [23]:
data

,uid,iid,r
0,0,0,8
1,0,1,10
2,0,2,10
3,0,3,1
4,0,4,6
...,...,...,...
536059,15037,2211,10
536060,15037,775,8
536061,15037,283,9
536062,15037,169,10


In [24]:
data_dict

NameError: name 'data_dict' is not defined

In [57]:
interacted_items = [item_encoder[i] for i in interacted_items]

In [15]:
x_train, x_valid, y_train, y_valid = train_test_split(data, data['uid'].values, test_size=0.2, shuffle=True,
                                                      stratify=data['uid'].values, random_state=42)

In [33]:
import pandas as pd
import torch
import numpy as np

import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from collections import OrderedDict
import pytorch_models.ngcf.model
from pytorch_models.ngcf.model import NGCF
from pytorch_models.ngcf.utility.parser import parse_args

import pytorch_models.ngcf.utility.metrics as metrics
from pytorch_models.ngcf.utility.load_data import *
from pytorch_models.ngcf.utility.batch_test import *

import pytorch_models.Data.daum.train_test_split as ts

n_users=15039, n_items=29713
n_interactions=536075
n_train=428860, n_test=107215, sparsity=0.00120
29713


In [58]:
class NgcfPredictor:
    def __init__(self):
        self.args = parse_args()
        self.device = 'cpu'
        self.num_recommendations = 10
        self.num_epochs = 10

    def sample(self, users, items):

        def sample_pos_items_for_u(u, num):
            # sample num pos items for u-th user
            pos_items = items
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num:
                    break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]

                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch

        def sample_neg_items_for_u(u, num):
            # sample num neg items for u-th user
            neg_items = []
            while True:
                if len(neg_items) == num:
                    break
                neg_id = np.random.randint(low=0, high=3706, size=1)[0]
                if (
                        neg_id not in items
                        and neg_id not in neg_items
                ):
                    neg_items.append(neg_id)
            return neg_items

        pos_items, neg_items = [], []
        for u in users:
            pos_items += sample_pos_items_for_u(u, 1)
            neg_items += sample_neg_items_for_u(u, 1)

        return users, pos_items, neg_items

    def predict(self, interacted_items):
        """ # 회원가입한 사용자 버전(새로 회원가입을 한 경우에는 유저 임베딩이 추가되기 때문에 이전에 배치 or 온라인 학습된 모델을 checkpoint라는 변수로 불러와서 유저의 수를 new_num_user 변수에 생성) -> 아직 고민해봐야함..
        checkpoint = torch.load('NGCF.pkl')
        checkpoint_dict = dict(OrderedDict(checkpoint))
        new_num_user = len(checkpoint_dict['feature_dict.user'])
        
        user_id = new_num_user + 1
        print(user_id)
        """
        print(f"interacted_items : {interacted_items}")
        # 비회원 데이터 적재 X
        # user_id = 10538
        user_id = len(ts.user_encoder) + 1
        item_num = len(ts.item_encoder)
        print(user_id)

        data_generator.train_items[user_id - 1] = interacted_items

        new_user_item_dst = interacted_items
        new_user_item_src = [user_id - 1] * len(new_user_item_dst)
        user_selfs = list(range(user_id))
        item_selfs = list(range(item_num))
        data_dict = {
            ("user", "user_self", "user"): (user_selfs, user_selfs),
            ("item", "item_self", "item"): (item_selfs, item_selfs),
            ("user", "ui", "item"): (
                data_generator.user_item_src + new_user_item_src,
                data_generator.user_item_dst + new_user_item_dst
            ),
            ("item", "iu", "user"): (
                data_generator.user_item_dst + new_user_item_dst,
                data_generator.user_item_src + new_user_item_src
            ),
        }
        num_dict = {"user": user_id, "item": item_num}

        print(num_dict)
        data_generator.g = dgl.heterograph(data_dict, num_nodes_dict=num_dict)

        model = NGCF(
            data_generator.g, 64, [64, 64, 64], [0.1, 0.1, 0.1], 1e-5).to(self.device)
        optimizer = optim.Adam(model.parameters(), lr=self.args.lr)

        for epoch in range(self.num_epochs):
            loss, mf_loss, emb_loss = 0.0, 0.0, 0.0
            users, pos_items, neg_items = self.sample(new_user_item_src, new_user_item_dst)
            u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings = model(
                data_generator.g, "user", "item", users, pos_items, neg_items
            )
            batch_loss, batch_mf_loss, batch_emb_loss = model.create_bpr_loss(
                u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings
            )
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            loss += batch_loss

        # torch.save(model.state_dict(),self.args.model_name)

        model.eval()
        rec_items = {}

        # all-item test
        item = torch.tensor(list(set(item_selfs) - set(new_user_item_dst))).to(self.device)
        u_g_embeddings, pos_i_g_embeddings, _ = model(
            data_generator.g, "user", "item", user_id - 1, item, []
        )
        rate_batch = (
            model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
        )
        _, top_indices = torch.topk(rate_batch.view(-1), self.num_recommendations)
        recommended_items = item[top_indices]

        rec = []

        for i in recommended_items:
            rec.append(i)

        rec_list = [item.item() for item in rec]

        recomm_result = []

        for i in rec_list:
            recomm_result.append(ts.item_decoder[i])

        print(f"ngcf recomm_result : {recomm_result}")
        return recomm_result

In [59]:
ngcf_predictor = NgcfPredictor()

In [60]:
ngcf_predictor.predict(interacted_items)

interacted_items : [1923, 1201, 1433, 1433, 1201, 1923, 1201, 738, 738, 706, 492, 1201, 1201, 1201, 1337, 1201, 1433, 3752, 1433, 1433, 1433, 1433, 1433, 1433, 1433, 1433, 1923, 1923, 1923, 725, 725, 725, 1433, 1433, 3752, 1433, 1433, 3752, 288, 1433, 706, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1433, 1201, 1201, 1201, 1433, 1433, 1433, 711, 711, 3921, 481, 481, 2011, 732, 174, 174, 174, 173, 949, 630, 860, 860, 236, 712, 3363, 1221, 241, 1901, 1901, 1901, 4003, 2011, 706, 711, 711, 2011, 636, 711, 481, 801, 481, 22, 207, 801, 1401, 3921, 227, 3921, 711, 711, 2011, 801, 711, 1401, 711, 801, 219, 3360, 2011, 2011, 3921, 711, 22, 711, 711, 2011, 2011, 2011, 712, 712, 3921, 711, 1401, 801, 2011, 711, 3921, 481, 647, 1245, 92, 3921, 636, 6, 6, 711, 242, 2011, 224, 1401, 801, 801, 1401, 636, 1259, 481, 224, 3921, 801, 732, 1401, 1259, 711, 312, 3921, 283, 227, 481, 481, 725, 483, 202, 2011, 711, 3921, 801, 801, 481, 3752, 219, 394, 405, 88, 212, 92, 481, 394, 1901, 481, 

[95554, 85306, 85475, 70191, 110556, 76872, 122542, 113686, 84000, 75640]

In [48]:
num_dict = {"user": user_id, "item": item_num}

In [49]:
num_dict

{'user': 15039, 'item': 29713}

In [51]:
item_decoder[175838]

KeyError: 175838

In [52]:
item_encoder[175838]

13681

In [ ]:
[ for _ in interacted_items]

In [46]:
ngcf_predictor.predict(interacted_items)

interacted_items : [42238, 43688, 53962, 53962, 43688, 42238, 43688, 42148, 42148, 54361, 54772, 43688, 43688, 43688, 63478, 43688, 53962, 59546, 53962, 53962, 53962, 53962, 53962, 53962, 53962, 53962, 42238, 42238, 42238, 62730, 62730, 62730, 53962, 53962, 59546, 53962, 53962, 59546, 67165, 53962, 54361, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 53962, 43688, 43688, 43688, 53962, 53962, 53962, 108595, 108595, 108944, 114329, 114329, 91662, 63847, 68593, 68593, 68593, 75146, 109512, 106472, 100359, 100359, 95915, 100237, 76467, 73003, 84040, 85378, 85378, 85378, 87215, 91662, 54361, 108595, 108595, 91662, 119688, 108595, 114329, 76325, 114329, 156628, 108632, 76325, 88598, 108944, 134130, 108944, 108595, 108595, 91662, 76325, 108595, 88598, 108595, 76325, 122749, 111293, 91662, 91662, 108944, 108595, 156628, 108595, 108595, 91662, 91662, 91662, 100237, 100237, 108944, 108595, 88598, 76325, 91662, 108595, 108944, 114329, 139666, 146758, 156930, 108944, 119688

DGLError: The given number of nodes of node type item must be larger than the max ID in the data, but got 29713 and 175838.

In [36]:
interacted_items = [42238, 43688, 53962, 53962, 43688, 42238, 43688, 42148, 42148, 54361, 54772, 43688, 43688, 43688, 63478, 43688, 53962, 59546, 53962, 53962, 53962, 53962, 53962, 53962, 53962, 53962, 42238, 42238, 42238, 62730, 62730, 62730, 53962, 53962, 59546, 53962, 53962, 59546, 67165, 53962, 54361, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 42238, 53962, 43688, 43688, 43688, 53962, 53962, 53962, 108595, 108595, 108944, 114329, 114329, 91662, 63847, 68593, 68593, 68593, 75146, 109512, 106472, 100359, 100359, 95915, 100237, 76467, 73003, 84040, 85378, 85378, 85378, 87215, 91662, 54361, 108595, 108595, 91662, 119688, 108595, 114329, 76325, 114329, 156628, 108632, 76325, 88598, 108944, 134130, 108944, 108595, 108595, 91662, 76325, 108595, 88598, 108595, 76325, 122749, 111293, 91662, 91662, 108944, 108595, 156628, 108595, 108595, 91662, 91662, 91662, 100237, 100237, 108944, 108595, 88598, 76325, 91662, 108595, 108944, 114329, 139666, 146758, 156930, 108944, 119688, 144854, 144854, 108595, 144096, 91662, 122091, 88598, 76325, 76325, 88598, 119688, 118907, 114329, 122091, 108944, 76325, 63847, 88598, 118907, 108595, 145342, 108944, 121137, 134130, 114329, 114329, 62730, 1847, 139236, 91662, 108595, 108944, 76325, 76325, 114329, 59546, 122749, 133709, 115401, 129156, 139606, 156930, 114329, 133709, 85378, 114329, 109169, 67256, 162386, 167244, 164918, 147080, 114329, 119688, 119688, 91662, 91662, 53962, 114329, 42148, 108595, 76325, 54361, 115280, 161375, 108944, 108944, 156628, 134130, 133193, 135762, 158260, 91662, 108944, 143289, 156628, 91662, 59546, 156930, 156628, 156628, 156628, 156628, 156628, 156628, 156628, 156628, 156628, 156628, 156628, 108944, 108944, 108944, 108944, 108944, 108944, 108944, 108944, 169329, 169329, 169329, 169329, 169329, 169329, 169329, 158260, 158260, 167244, 167244, 167244, 167244, 156628, 150473, 150473, 150473, 150473, 150473, 150473, 150473, 108595, 108595, 108595, 108595, 108595, 91662, 91662, 91662, 91662, 108944, 169328, 144068, 150473, 150473, 161233, 164114, 159263, 169137, 175838, 76325, 91662, 169328, 158260, 163887, 119688, 83155, 76325, 91662, 91662, 156628, 108944, 156397, 164114]

In [37]:
user_id = len(ts.user_encoder) + 1
item_num = len(ts.item_encoder)
print(user_id)

data_generator.train_items[user_id - 1] = interacted_items

new_user_item_dst = interacted_items
new_user_item_src = [user_id - 1] * len(new_user_item_dst)
user_selfs = list(range(user_id))
item_selfs = list(range(item_num))
data_dict = {
    ("user", "user_self", "user"): (user_selfs, user_selfs),
    ("item", "item_self", "item"): (item_selfs, item_selfs),
    ("user", "ui", "item"): (
        data_generator.user_item_src + new_user_item_src,
        data_generator.user_item_dst + new_user_item_dst
    ),
    ("item", "iu", "user"): (
        data_generator.user_item_dst + new_user_item_dst,
        data_generator.user_item_src + new_user_item_src
    ),
}

15039


In [38]:
data_dict.keys()

dict_keys([('user', 'user_self', 'user'), ('item', 'item_self', 'item'), ('user', 'ui', 'item'), ('item', 'iu', 'user')])

In [45]:
len(new_user_item_src)

286

In [ ]:
data_dict

In [40]:
len(data_generator.user_item_src + new_user_item_src)

429146

In [41]:
len(data_generator.user_item_dst + new_user_item_dst)

429146

In [42]:
len(data_generator.user_item_dst + new_user_item_dst)

429146

In [43]:
len(data_generator.user_item_src + new_user_item_src)

429146